<a href="https://colab.research.google.com/github/zih-yan/abc1/blob/main/EX04_06_ET_Today_%E6%96%B0%E8%81%9E%E6%A8%99%E9%A1%8C%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 問題定義



*   爬取過去一年 ET Today 的新聞標題做為資料集。

*   使用 Jieba 進行斷詞。
*   使用單純貝氏演算法進行新聞標題分類。



## 資料收集

In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-ettoday_news.csv'
df = pd.read_csv(url)
df.head()

,類別,標題
0,政治,青年座談不被AI取代的關鍵力 葉丙成：只有這兩種人才能真正把握時間紅利
1,政治,蕭美琴直播初體驗曝「咪琴嚴選」 邀網友挺花蓮
2,政治,疑似國軍「在美受訓」照曝光！ 陸軍不評論
3,政治,「蔡英文、柯文哲有談NCC」 府：沒談人選、政黨比例更沒私下喬
4,政治,總統府否認蔡英文向柯喬NCC委員 黃國昌嗆：醜聞被揭露還敢說謊


## 資料前處理

### 資料清理

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29216 entries, 0 to 29215
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   類別      29216 non-null  object
 1   標題      29216 non-null  object
dtypes: object(2)
memory usage: 456.6+ KB


### 探索性分析

### 資料分割

In [3]:
!wget -O dict.txt https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-dict.txt
!wget -O stopwords.txt https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-stopwords.txt

--2025-06-11 19:50:05--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-dict.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4154480 (4.0M) [application/octet-stream]
Saving to: ‘dict.txt’

dict.txt            100%[===================>]   3.96M  --.-KB/s    in 0.08s   

2025-06-11 19:50:06 (47.8 MB/s) - ‘dict.txt’ saved [4154480/4154480]

--2025-06-11 19:50:06--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8129 (7.9

In [4]:
import jieba
# 加入繁體中文字典
jieba.set_dictionary('dict.txt')
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = set(line.strip() for line in f if line.strip())
    return stopwords
# 分詞並過濾停用詞
def jieba_cut_with_stopwords(text, stopwords):
    words = jieba.cut(text)
    return [word for word in words if word not in stopwords and word.strip() != '']
# 使用範例
stopwords = load_stopwords("stopwords.txt")

In [5]:
df

,類別,標題
0,政治,青年座談不被AI取代的關鍵力 葉丙成：只有這兩種人才能真正把握時間紅利
1,政治,蕭美琴直播初體驗曝「咪琴嚴選」 邀網友挺花蓮
2,政治,疑似國軍「在美受訓」照曝光！ 陸軍不評論
3,政治,「蔡英文、柯文哲有談NCC」 府：沒談人選、政黨比例更沒私下喬
4,政治,總統府否認蔡英文向柯喬NCC委員 黃國昌嗆：醜聞被揭露還敢說謊
...,...,...
29211,生活,瞞未婚夫下海拍片 休息3個月後「I級AV女優宣布引退」！原因曝
29212,生活,努力考上清大！家人1句「能嫁有錢工程師了」讓她心冷
29213,生活,北榮藝廊快閃一週 陳威明：鎮館之寶《古柏園》象徵醫者精神
29214,生活,收到「一整串粽子」媽氣炸！大票愣「每年都吃到吐」：沒聽過


In [6]:
x, y = df['標題'], df['類別']

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### 類別轉換

In [8]:
from sklearn.preprocessing import LabelEncoder
lebel_encoder = LabelEncoder()

y_train = lebel_encoder.fit_transform(y_train)
y_test = lebel_encoder.transform(y_test)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

### 特徵縮放

## 模型訓練

In [14]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB(alpha=1.0) # alpha 就是設定如果該詞不存在時，因貝氐運算是採用乘法，所以用 1
mnb.fit(x_train, y_train)

MultinomialNB()

In [15]:
mnb.score(x_train, y_train)

0.9567858976553141

## 模型評估

In [18]:
from sklearn.metrics import accuracy_score
y_pred = mnb.predict(x_test)           # 用模型預測測試集
accuracy = accuracy_score(y_test, y_pred)  # 計算準確率
print("Accuracy:", accuracy)

Accuracy: 0.3073237508555784


## 模型調整

## 模型部署

### 儲存模型

### 推論預測

In [22]:
text = '青年座談不被AI取代的關鍵力　葉丙成：只有這兩種人才能真正把握時間紅利'
s = vectorizer.transform([' '.join(jieba_cut_with_stopwords(text, stopwords))])
c = mnb.predict(s)[0]
category = lebel_encoder.classes_[c]
print(category)

財經
